In [46]:
import wandb
from tokenizers import Tokenizer
from model import GoePT
import json
import os
import cupy as cp
import torch
import numpy as np

wandb.init(mode="disabled")


checkpoint_filename = "bias_fix.json"
with open(
    os.path.join("../../checkpoints", checkpoint_filename),
    mode="r",
    encoding="utf-8",
) as in_file:
    state_dict = json.load(in_file)
model_loaded = GoePT.from_state_dict(state_dict)



tokenizer: Tokenizer = Tokenizer.from_file("../tokenizers/goe_pt/goe_pt_tokenizer.json")
# tokenizer.enable_padding(length=256)




In [52]:
start_text = """MEPHISTOPHELES:
am Throne kniend.
Was ist verwünscht und stets willkommen?
Was ist ersehnt und stets verjagt?
Was immerfort in Schutz genommen?
Was hart gescholten und verklagt?"""
text = start_text
print(start_text,end="")
for i in range(100):
    tokenized = tokenizer.encode(text)
    #print(tokenized.ids)
    padded_tokenized = cp.array(tokenized.ids)
    tokenized = padded_tokenized.reshape((1, -1))
    #print(tokenized)
    logits, _ = model_loaded.forward(tokenized)
    #print(logits)
    probabilites ,out = torch.topk(torch.tensor(logits),10,sorted=True)
    out = out.cpu().numpy().flatten().tolist()
    probabilites = probabilites.cpu().flatten()
    probs = torch.nn.functional.softmax(probabilites,-1)
    selected = np.random.choice(out, size=1, p=probs).tolist()

    #print(out)

    #to_remove = [80,8,10,3,17]
    #for unimportant in to_remove:
    #    try:
    #        selected.remove(unimportant)
    #    except:
    #        pass
    out = [tokenizer.decode((val,)) for val in selected]
    #print(out)
    text = text + out[0]
    print(out[0],end="")
# 0: start, 1: end

MEPHISTOPHELES:
am Throne kniend.
Was ist verwünscht und stets willkommen?
Was ist ersehnt und stets verjagt?
Was immerfort in Schutz genommen?
Was hart gescholten und verklagt?.

Und, wenn!
FAUST. Die
MEPHISTOPHELES ich, der; doch, was sich. Du
Der und nicht mehr und, der;
Der' zu.
Der ich nicht so,
Und ist das Herz:'ge dich'!
MEPHISTOPHELES:s nicht mehr;

Und wenn!  Er: das. Du!  und die: der Welt!, wenn, der. (nach, der sich nicht so.


In [7]:

logits, _ = model_loaded.forward(tokenized)


print(logits)
out = torch.topk(torch.tensor(logits),10,sorted=True)[1]
out = out.cpu().numpy().flatten()
print(out)
out = [tokenizer.decode((val,)) for val in out]
print(out)



[[[-1.72611997e-03 -1.78618122e-03  1.22140817e-04 ...  3.81668196e-05
   -5.32563662e-04 -1.92076851e-04]]]
[  80    8   10    3 1344  134  389  181   17  139]
['\n', ',', '.', '!', ' Du', ' und', ' Er', ' ist', ':', ' ich']
